In [ ]:
'''
Usage:
1. Data directory must be in the same directory as the script
2. Images, lables and superpixels directories must be inside data directory
3. Script parameters must be specified in script parameters cell
'''

In [26]:
import numpy as np
import os
import collections
import matplotlib.pyplot as plt
import skimage.future.graph as skigraph
import shutil
import pickle
import keras
import tensorflow as tf
import networkx as nx
import time
import sys

from kgcnn.literature.GCN import make_model
from kgcnn.utils.data import ragged_tensor_from_nested_numpy
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN

In [27]:
# config

# script parameterss
MAIN_DIR_PATH = "/home/daniel/Praca inżynierska/DetectingInflammationsInImagesOfSacroiliacJoint"
DATA_DIR_NAME = "data"
IMAGES_DIR_NAME = "images"
SUPERPIXELS_LABELS_DIR_NAME = "labels"
SUPERPIXELS_DIR_NAME = "superpixels"

IMAGE_FILE_PREFIX = "STIR"
SUPERPIXELS_FILE_PREFIX = "1000SuperPixelMeanValueTPS_"

IMG_CMAP = "pink"
EDGE_CMAP = "viridis"
EDGE_WIDTH = 2

CREATE_SUPERPIXELS_IMAGES = False

# globals
DATA_DIR_PATH = f"{MAIN_DIR_PATH}/{DATA_DIR_NAME}"
IMAGES_DIR_PATH = f"{DATA_DIR_PATH}/{IMAGES_DIR_NAME}"
SUPERPIXELS_LABELS_DIR_PATH = f"{DATA_DIR_PATH}/{SUPERPIXELS_LABELS_DIR_NAME}"
SUPERPIXELS_DIR_PATH = f"{DATA_DIR_PATH}/{SUPERPIXELS_DIR_NAME}"

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(suppress=True)

In [28]:
# utils
def show_2d_image_from_3d_image(image_3d, index_of_2d_image):
    plt.figure(figsize = (20,20))
    plt.imshow(image_3d[:,:,index_of_2d_image], cmap='gray')
    plt.show()
    
def save_object(filename, obj):
    obj_file = open(filename, 'wb')
    pickle.dump(obj, obj_file)
    obj_file.close()
    
def load_object(filename):
    obj_file = open(filename, 'rb')
    obj = pickle.load(obj_file)
    obj_file.close()
    
    return obj

# read data
def read_binary_data_3D(file_name, image_size, image_count, voxel_bytes, signed='Y', byte_order='BE'):
    if voxel_bytes == 2:
        if signed == 'N':
            d = np.zeros((image_size, image_size, image_count), np.uint16)
        else:
            d = np.zeros((image_size, image_size, image_count), np.int16)
    elif voxel_bytes == 1:
        if signed == 'N':
            d = np.zeros((image_size, image_size, image_count), np.uint8)
        else:
            d = np.zeros((image_size, image_size, image_count), np.int8)
    else:
        print('Wrong number of bytes per voxel')
        return

    f = open(file_name, "rb")
    for i in range(0, image_count):
        for j in range(0, image_size):
            for k in range(0, image_size):
                byte = f.read(voxel_bytes)
                if voxel_bytes == 2:
                    if byte_order == 'BE':
                        a = 256 * byte[0] + byte[1]
                    else:
                        a = byte[0] + 256 * byte[1]
                else:
                    a = byte[0]
                d[j,k,i] = a
    f.close()
    return d

# rag creation
FileInfo = collections.namedtuple('FileInfo', ['name', 'patient_id', 'height', 'width', 'image_count', 'voxel_bytes'])

def create_rag_on_superpixels_image(superpixel_labels, superpixels, rag, file_info, image_number, edge_width, img_cmap, edge_cmap):
    PATIENT_ID_DIR_PATH = f"{DATA_DIR_PATH}/rags/{file_info.patient_id}"
    RAG_IMAGE_PATH = f"{PATIENT_ID_DIR_PATH}/rag_{image_number}_{file_info.height}_{file_info.width}_{file_info.image_count}_{file_info.voxel_bytes}_.png"
    if os.path.exists(RAG_IMAGE_PATH):
        shutil.rmtree(PATIENT_ID_DIR_PATH)
    os.makedirs(PATIENT_ID_DIR_PATH, exist_ok=True)
    
    rag_file = open(RAG_IMAGE_PATH, "w")
    rag_file.close()
    
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(20, 20))
    ax.axis('off')
    lc = skigraph.show_rag(
        superpixel_labels, rag, superpixels, img_cmap=img_cmap, edge_width=edge_width, edge_cmap=edge_cmap, ax=ax
    )
    fig.colorbar(lc, fraction=0.04, ax=ax)
    fig.savefig(RAG_IMAGE_PATH)
    plt.close(fig) 
    
def assign_label(image, superpixels_labels, rag):
    white_pixels_count = {}
    total_pixels_count = {}
    for (i, row) in enumerate(superpixels_labels):
        for (j, label) in enumerate(row):
            if label in total_pixels_count:
                total_pixels_count[label] += 1
            else:
                total_pixels_count[label] = 1 
            if image[i][j] >= 130:
                if label in white_pixels_count:
                    white_pixels_count[label] += 1
                else:
                    white_pixels_count[label] = 1
                    
    for (i, node) in enumerate(rag):
        if i in white_pixels_count and white_pixels_count[i] / total_pixels_count[i] >= 0.75:
            rag.nodes[node]['label'] = '1'
        else:
            rag.nodes[node]['label'] = '0'
            
def process_3D_image(superpixels_labels_file, image_file, superpixels_file, file_info, create_superpixels_images = CREATE_SUPERPIXELS_IMAGES):
    rags = []
    
    for file_index in range(file_info.image_count):
        print(f"Processing sub images: {file_index + 1}/{file_info.image_count}")
        superpixels_labels = superpixels_labels_file[:,:,file_index]
        img = image_file[:,:,file_index]
        img = img/(img.max()/255.0)     
        rag = skigraph.rag_mean_color(img, superpixels_labels)
        rags.append(rag)
        assign_label(img, superpixels_labels, rag)
        superpixels = superpixels_file[:,:,file_index]
        
        if create_superpixels_images:
            create_rag_on_superpixels_image(
               superpixels_labels, superpixels, rag, file_info, file_index + 1, EDGE_WIDTH, IMG_CMAP, EDGE_CMAP
            )
        
    return rags

def process_all_3D_images_from_directory(images_dir_path, superpixels_labels_dir_path, superpixels_dir_path):
    file_names = os.listdir(superpixels_labels_dir_path)
    rags = {i: list() for i in range(1, len(file_names) + 1)}

    for (file_3d_index, file_name) in enumerate(file_names, start=1):
        unprocessed_file_info = file_name.split("_")[:-1]
        unprocessed_file_info[1:] = [int(unprocessed_file_info[i]) for i in range(1, len(unprocessed_file_info))]
        file_info = FileInfo(*unprocessed_file_info)
        file_suffix = f"{file_info.patient_id}_{file_info.height}_{file_info.width}_{file_info.image_count}_{file_info.voxel_bytes}_.raw"
        print(f"Directory processing progress: {file_3d_index}/{len(file_names)}, current patient: {file_info.patient_id}")
        
        superpixels_labels_file = read_binary_data_3D(
            f"{superpixels_labels_dir_path}/{file_name}",
            file_info.height,
            file_info.image_count,
            file_info.voxel_bytes,
            signed='Y',
            byte_order='EB'
        )
        image_file = read_binary_data_3D(
            f"{images_dir_path}/{IMAGE_FILE_PREFIX}_{file_suffix}",
            file_info.height,
            file_info.image_count,
            file_info.voxel_bytes,
            signed='Y',
            byte_order='BE'
        )
        superpixels_file = read_binary_data_3D(
            f"{superpixels_dir_path}/{SUPERPIXELS_FILE_PREFIX}{file_suffix}",
            file_info.height,
            file_info.image_count,
            file_info.voxel_bytes,
            signed='Y',
            byte_order='EB'
        )
        
        rags[file_info.patient_id] = process_3D_image(superpixels_labels_file, image_file, superpixels_file, file_info)
    
    print("All 3D files have been processed")
    return rags

# split into subgraphs containing node and node neighbors
def split_into_subgraphs(rags, how_many):
    neighbor_graphs = []
    labels = []
    for (i, rag_3d) in enumerate(rags.values()):
        if i ==  how_many:
            break
        print(i)
        for rag_2d in rag_3d:
            for node in rag_2d.nodes:
                nodes = [n for n in rag_2d.neighbors(node)]
                nodes.append(node)
                neighbor_graphs.append(rag_2d.subgraph(nodes))
                labels.append([float(rag_2d.nodes[node]['label'])])
    return neighbor_graphs, labels

def prepare_data(neighbor_graphs):
    nodes = []
    edge_indices = []
    edges = []
    for graph in neighbor_graphs:
        mean_intensities = []
        edges_prim = []
        for node in graph.nodes:
            mean_intensities.append(graph.nodes[node]['mean color'][0])  
        for data in graph.edges.data():
            edges_prim.append(data[2]['weight'])
        nodes.append(mean_intensities)
        edges.append(edges_prim)
        edge_indices.append(normalize_edge_indices([list(e) for e in graph.edges]))
    return nodes, edge_indices, edges

def normalize_edge_indices(edge_indices):
    result = []
    flat_list = []
    for sublist in edge_indices:
        for item in sublist:
            flat_list.append(item)
            
    flat_list.sort()
    flat_list = list(dict.fromkeys(flat_list))
    change = {key:value for (value, key) in enumerate(flat_list)}
    
    for sublist in edge_indices:
        temp = []
        for item in sublist:
            temp.append(change[item])
        result.append(temp)    
    
    
    return result

In [23]:
rags = load_object('saved_objects/rags')
neighbor_graphs = load_object('saved_objects/neighbor_graphs')
nodes = load_object('saved_objects/nodes')
edges = load_object('saved_objects/edges')
edge_indices = load_object('saved_objects/edge_indices')
labels = load_object('saved_objects/labels')
model = keras.models.load_model('/home/daniel/Praca inżynierska/DetectingInflammationsInImagesOfSacroiliacJoint/saved_objects')

In [ ]:
rags = process_all_3D_images_from_directory(IMAGES_DIR_PATH, SUPERPIXELS_LABELS_DIR_PATH, SUPERPIXELS_DIR_PATH)

Directory processing progress: 1/42, current patient: 12
Processing sub images: 1/18
Processing sub images: 2/18
Processing sub images: 3/18
Processing sub images: 4/18
Processing sub images: 5/18
Processing sub images: 6/18
Processing sub images: 7/18
Processing sub images: 8/18
Processing sub images: 9/18
Processing sub images: 10/18
Processing sub images: 11/18
Processing sub images: 12/18
Processing sub images: 13/18
Processing sub images: 14/18
Processing sub images: 15/18
Processing sub images: 16/18
Processing sub images: 17/18
Processing sub images: 18/18
Directory processing progress: 2/42, current patient: 8
Processing sub images: 1/24
Processing sub images: 2/24
Processing sub images: 3/24
Processing sub images: 4/24
Processing sub images: 5/24
Processing sub images: 6/24
Processing sub images: 7/24
Processing sub images: 8/24
Processing sub images: 9/24
Processing sub images: 10/24
Processing sub images: 11/24
Processing sub images: 12/24
Processing sub images: 13/24
Proces

In [ ]:
neighbor_graphs, labels = split_into_subgraphs(rags, 7)

In [ ]:
nodes, edge_indices, edges =  prepare_data(neighbor_graphs)

In [24]:
for i in range(len(labels)):
    edges[i] = np.expand_dims(edges[i], axis=-1).astype(np.float32)
for i in range(len(labels)):
    nodes[i] =np.expand_dims(nodes[i],axis=-1).astype(np.int)
    
labels_train, labels_test, nodes_train, nodes_test, edges_train, edges_test, edge_indices_train, edge_indices_test = train_test_split(labels, nodes, edges, edge_indices,  train_size=0.8, random_state=1)

nodes_train = ragged_tensor_from_nested_numpy(nodes_train)
edges_train = ragged_tensor_from_nested_numpy(edges_train)
edge_indices_train = ragged_tensor_from_nested_numpy(edge_indices_train)

nodes_test = ragged_tensor_from_nested_numpy(nodes_test)
edges_test = ragged_tensor_from_nested_numpy(edges_test)
edge_indices_test = ragged_tensor_from_nested_numpy(edge_indices_test)

xtrain = nodes_train, edges_train, edge_indices_train
xtest = nodes_test, edges_test, edge_indices_test
ytrain = np.array(labels_train)
ytest = np.array(labels_test)

In [25]:
model = make_model(
    name = "Unet",
    inputs = [{'shape': (None, 1), 'name': "node_attributes", 'dtype': 'float32', 'ragged': True},
            {'shape': (None, 1), 'name': "edge_attributes", 'dtype': 'float32', 'ragged': True},
            {'shape': (None, 2), 'name': "edge_indices", 'dtype': 'int64', 'ragged': True}],
    input_embedding = {"node": {"input_dim": 55, "output_dim": 64},
                       "edge": {"input_dim": 10, "output_dim": 64}},

    output_embedding =  'graph',
    output_mlp = {"use_bias": [True, True, False], "units": [140, 70, 1],
                "activation": ['relu', 'relu', 'sigmoid']},
    gcn_args = {"units": 64, "use_bias": True, "activation": 'relu', "pooling_method": 'mean', 
                "normalize_by_weights": False},
)

# Set learning rate and epochs
learning_rate_start = 1e-3
learning_rate_stop = 1e-4
epo = 150
epomin = 100
epostep = 10

# Compile model with optimizer and loss
optimizer = tf.keras.optimizers.Adam(lr=learning_rate_start)
cbks = LinearLearningRateScheduler(learning_rate_start, learning_rate_stop, epomin, epo)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              weighted_metrics=['accuracy'])
print(model.summary())

# Start and time training
start = time.process_time()
hist = model.fit(xtrain, ytrain,
                 epochs=epo,
                 batch_size=32,
                 callbacks=[cbks],
                 validation_freq=epostep,
                 validation_data=(xtest, ytest),
                 verbose=2
                 )
stop = time.process_time()
print("Print Time for taining: ", stop - start)

# Get loss from history
trainlossall = np.array(hist.history['accuracy'])
testlossall = np.array(hist.history['val_accuracy'])
acc_valid = testlossall[-1]

# Plot loss vs epochs
plt.figure()
plt.plot(np.arange(trainlossall.shape[0]), trainlossall, label='Training ACC', c='blue')
plt.plot(np.arange(epostep, epo + epostep, epostep), testlossall, label='Test ACC', c='red')
plt.scatter([trainlossall.shape[0]], [acc_valid], label="{0:0.4f} ".format(acc_valid), c='red')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Interaction Network Loss')
plt.legend(loc='upper right', fontsize='x-large')
plt.show()

INFO:kgcnn: Updated model kwargs:
{'depth': 3,
 'gcn_args': {'activation': 'relu',
              'has_unconnected': True,
              'is_sorted': False,
              'normalize_by_weights': False,
              'pooling_method': 'mean',
              'units': 64,
              'use_bias': True},
 'input_embedding': {'edge': {'input_dim': 10, 'output_dim': 64},
                     'node': {'input_dim': 55, 'output_dim': 64}},
 'inputs': [{'dtype': 'float32',
             'name': 'node_attributes',
             'ragged': True,
             'shape': (None, 1)},
            {'dtype': 'float32',
             'name': 'edge_attributes',
             'ragged': True,
             'shape': (None, 1)},
            {'dtype': 'int64',
             'name': 'edge_indices',
             'ragged': True,
             'shape': (None, 2)}],
 'name': 'Unet',
 'output_embedding': 'graph',
 'output_mlp': {'activation': ['relu', 'relu', 'sigmoid'],
                'units': [140, 70, 1],
                '

/home/daniel/anaconda3/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


3786/3786 - 9s - loss: 109.3959 - accuracy: 0.9391
Epoch 2/150
3786/3786 - 10s - loss: 0.0787 - accuracy: 0.9835
Epoch 3/150
3786/3786 - 10s - loss: 2.9409 - accuracy: 0.9837
Epoch 4/150
3786/3786 - 10s - loss: 2.5198 - accuracy: 0.9843
Epoch 5/150
3786/3786 - 10s - loss: 0.0743 - accuracy: 0.9854
Epoch 6/150
3786/3786 - 10s - loss: 0.0549 - accuracy: 0.9855
Epoch 7/150
3786/3786 - 10s - loss: 0.0546 - accuracy: 0.9855
Epoch 8/150
3786/3786 - 10s - loss: 0.6718 - accuracy: 0.9853
Epoch 9/150
3786/3786 - 9s - loss: 0.0541 - accuracy: 0.9857
Epoch 10/150
3786/3786 - 11s - loss: 17.9435 - accuracy: 0.9822 - val_loss: 0.0551 - val_accuracy: 0.9857
Epoch 11/150
3786/3786 - 10s - loss: 4.1984 - accuracy: 0.9852
Epoch 12/150
3786/3786 - 10s - loss: 0.0542 - accuracy: 0.9858
Epoch 13/150
3786/3786 - 10s - loss: 0.6128 - accuracy: 0.9855
Epoch 14/150
3786/3786 - 10s - loss: 0.0538 - accuracy: 0.9857
Epoch 15/150
3786/3786 - 10s - loss: 4.7533 - accuracy: 0.9833
Epoch 16/150
3786/3786 - 10s - lo

KeyboardInterrupt: 

In [21]:
probability = model.predict(xtest)
predictions = np.round(probability)
comparison = np.concatenate((predictions, ytest, probability), axis=1)
print(comparison)

[[0.         0.         0.        ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.02784356]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.        ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         1.         0.5       ]
 [0.         0.         0.        ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.        ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.5       ]
 [0.         0.         0.        ]
 [0.         0.         0.  

In [12]:
save_object('saved_objects/rags', rags)
save_object('saved_objects/neighbor_graphs', neighbor_graphs)
save_object('saved_objects/labels', labels)
save_object('saved_objects/nodes', nodes)
save_object('saved_objects/edges', edges)
save_object('saved_objects/edge_indices', edge_indices)
model.save('/home/daniel/Praca inżynierska/DetectingInflammationsInImagesOfSacroiliacJoint/saved_objects')